In [356]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

from set_init import *
from dlbond.utils.split import Splitter
from dlbond.utils.cluster import cluster_KMeans_base

In [145]:
def plot_target_hist(y_train, y_test):
    y_train_ = y_train.reset_index()
    y_train_["type"] = "train"
    y_test_ = y_test.reset_index()
    y_test_["type"] = "test"
    df = pd.concat([y_train_, y_test_])
    df.columns = ["Date", "target", "type"]

    fig = px.histogram(
        df, 
        x="target", 
        color="type", 
        marginal="box", 
        title="訓練データとテストデータの目的変数のヒストグラム",            
        barmode="overlay",
        histnorm="probability",
        )
    return fig

def plot_previous(y_train, y_test):
    def combine_yesterday(y):
        df = pd.concat([y, y.shift(-1)], axis=1).dropna().reset_index()
        df.columns = ["Date", "当日", "前営業日"]
        
        return df

    df_train_ = combine_yesterday(y_train)
    df_test_ = combine_yesterday(y_test)
    df_train_["type"] = "train"
    df_test_["type"] = "test"
    df = pd.concat([df_train_, df_test_])

    fig = px.scatter(
        df, 
        x="前営業日", 
        y="当日", 
        color="type", 
        title="前日と当日の目的変数の散布図",        
        marginal_x="histogram",
        marginal_y="histogram",       
        trendline='ols', 
        )
    return fig

In [146]:
sp = Splitter(dfs_input, config)

start_date_train = dt.datetime(2015, 1, 1)
end_date_train = dt.datetime(2015, 12, 31)
start_date_test = dt.datetime(2016, 1, 1)
end_date_test = dt.datetime(2016, 12, 31)
X_train, y_train, X_test, y_test, _ = sp.make_dataset(start_date_train, end_date_train, start_date_test, end_date_test)

figs = []
figs.append(plot_target_hist(y_train, y_test))
figs.append(plot_previous(y_train, y_test))

In [147]:
for fig in figs:
    fig.show()

In [120]:
n_components = 16
pca = PCA(n_components=n_components, random_state=0)        
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_pca = pca.fit_transform(X_train_scaled)
pca_name = ["PC"+str(i+1) for i in range(len(pca.explained_variance_ratio_))]
pca_explained_variance_ratio = pd.Series(pca.explained_variance_ratio_, index=pca_name)

In [119]:

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
           x=pca_explained_variance_ratio.index, 
           y=pca_explained_variance_ratio.values, 
           name="寄与率")
    )
fig.add_trace(
       go.Scatter(
              x=pca_explained_variance_ratio.index, 
              y=pca_explained_variance_ratio.cumsum(),
              name="累積寄与率"
              ),
       secondary_y=True,       
       )


In [188]:
corr_distance, clusters, silh = cluster_KMeans_base(
    corr_distance=X_train_scaled.corr(),
    min_num_clusters=3,
    max_num_clusters=10,
    n_init=5,
    random_state=0)

cols = []
for key, val in clusters.items():
    cols += val
    
corr_info = pd.DataFrame([corr_distance.index, cols], index=["cluster", "feature"]).T
corr_distance.columns = cols
corr_distance.index = cols

corr_num = np.zeros((corr_distance.shape[0], corr_distance.shape[1], 2))
for i in range(corr_num.shape[0]):
    for j in range(corr_num.shape[1]):
        corr_num[j][i][0] = str(corr_info.iloc[i,0])
        corr_num[i][j][1] = str(corr_info.iloc[i,0])

####### cluster3 #######
####### cluster7 #######
####### cluster6 #######
####### cluster9 #######


In [248]:
fig = go.Figure(data=go.Heatmap(
    x=corr_distance.columns,
    y=corr_distance.columns,
    z=corr_distance,
    customdata=corr_num,
    hovertemplate='<b>corr:%{z:.3f}</b><br>feature1:%{x}</b><br><b>feature2:%{y}</b><br>cluster1:%{customdata[0]}</b><br>cluster2:%{customdata[1]}',
    hoverongaps = False,
    name="",
    )
)
fig.update_xaxes(side="top")
fig.show()

In [377]:
pc_all = pd.DataFrame(pca.components_, columns=X_train.columns, index=pca_name).T
extract_names = ["PC1", "PC2"]
pc_extracted = pc_all[extract_names].reset_index()
pc_extracted.columns = ["feature"] + extract_names
corr_info_reindex =corr_info.set_index("feature").loc[list(pc_extracted["feature"])]
corr_info_reindex = corr_info_reindex.reset_index()
pc_extracted["cluster"] = corr_info_reindex.values[:,1].astype(str)

In [362]:
def plot_pc_scatter(pc_extracted):
    fig = px.scatter(
        pc_extracted,
        x="PC1",
        y="PC2",
        color="cluster",
        hover_data=["cluster", "feature"],    
        )
    fig.update_layout(
        xaxis={'title': 'PC1'},
        yaxis={'title': 'PC2'},
        hovermode="closest",
        title="第一主成分と第二主成分における観測変数の寄与度"
        )
    
    return fig

In [381]:
def plot_corr_tSNE(
    corr_distance, 
    corr_info_reindex,
    n_components=2,
    learning_rate=300,
    perplexity=30,
    early_exaggeration=12,
    init='random',
    random_state=0
    ):
    tSNE = TSNE(n_components=n_components, learning_rate=learning_rate, \
                perplexity=perplexity, early_exaggeration=early_exaggeration, \
                init=init, random_state=random_state)

    tSNE_extracted = tSNE.fit_transform(corr_distance)
    tSNE_extracted = pd.DataFrame(data=tSNE_extracted, columns=["first",  "second"])
    tSNE_extracted["cluster"] = corr_info_reindex.values[:,1].astype(str)
    tSNE_extracted["feature"] = corr_distance.columns

    fig = px.scatter(
        tSNE_extracted,
        x="first",
        y="second",
        color="cluster",
        hover_data=["cluster", "feature"],    
        )
    fig.update_layout(
        xaxis={'title': 'first'},
        yaxis={'title': 'second'},
        hovermode="closest",
        title="t-SNEによる相関係数行列の次元削減"
        )

    return fig

fig = plot_corr_tSNE(corr_distance, corr_info_reindex)
fig.show()